In [1]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
#read csv files

airlines_df= pd.read_csv('Resources/airlines.csv', low_memory=False)
flights_df = pd.read_csv('Resources/flights.csv', low_memory=False)

In [ ]:
airlines_df.info()

In [ ]:
#show first 5 records
airlines_df.head()


In [ ]:
flights_df.info()

In [ ]:
#determine the number of nulls per column
flights_df.isnull().sum()

In [ ]:
#drop unneeded columns
airline_data_dropped_df=flights_df.drop(columns=['TAIL_NUMBER', 'WHEELS_ON', 'WHEELS_OFF', 'TAXI_IN','TAXI_OUT', 'AIR_TIME'])

In [ ]:
airline_data_dropped_df.info()

In [ ]:
#merge the year, month and day columns into one Date column
airline_data_dropped_df['DATE'] = pd.to_datetime(airline_data_dropped_df[['YEAR', 'MONTH', 'DAY']])

#drop the individual year, month and day columns
airline_data_dropped_df = airline_data_dropped_df.drop(['YEAR', 'MONTH', 'DAY'], axis=1)

In [ ]:
#function to determine if flight was on time or delayed
def delay_status(arrival_depature_time):
    flight_status=''
    if arrival_depature_time  > 15:
        # Extract 'name' and 'value'
        flight_status='DELAYED'
    else:
        flight_status='ON_TIME'
        
    return flight_status

#assign depature flight status (delayed or ontime) to new column
airline_data_dropped_df['DEPATURE_DELAY_STATUS']=airline_data_dropped_df['DEPARTURE_DELAY'].apply(delay_status)


#assign arrival flight status (delayed or ontime) to new column
airline_data_dropped_df['ARRIVAL_DELAY_STATUS']=airline_data_dropped_df['ARRIVAL_DELAY'].apply(delay_status)

In [ ]:
airline_data_dropped_df.head()

In [ ]:
#set index to the Date column
airline_data_dropped_df=airline_data_dropped_df.set_index('DATE', drop=True)

#slice and get data from January to April
first_quarter_df = airline_data_dropped_df['2015-01-01':'2015-04-30']

In [ ]:
first_quarter_df=first_quarter_df.rename(columns={"AIRLINE": "AIRLINE_CODE"})

In [ ]:
first_quarter_df.info()

In [ ]:
#drop rows with null depature delays
first_quarter_df.dropna(subset=['DEPARTURE_DELAY'], inplace=True)

In [ ]:
#get all the delayed depatures
delayed_depatures=first_quarter_df[first_quarter_df['DEPATURE_DELAY_STATUS']=='DELAYED']

#get day of the week for all delayed depatures
day_of_week_depature_df=pd.DataFrame(delayed_depatures[["DAY_OF_WEEK"]])

#count all delayed depatures and group them by day of the week
depatures_day_of_week_delays_df=pd.DataFrame(day_of_week_depature_df.groupby(['DAY_OF_WEEK']).value_counts())

#reset index
depatures_day_of_week_delays_df=depatures_day_of_week_delays_df.reset_index('DAY_OF_WEEK')

#function to get the day name from the day number
def get_day_name(day_number):
    day_name=''
    
    match day_number:
        case 1:
            day_name="Monday"
        case 2:
            day_name="Tuesday"
        case 3:
            day_name="Wednesday"
        case 4:
            day_name="Thursday"
        case 5:
            day_name="Friday"
        case 6:
            day_name="Saturday"
        case 7:
            day_name="Sunday"
    
    return day_name

#get the name of the day of the week
depatures_day_of_week_delays_df["DAY_NAME"]= depatures_day_of_week_delays_df['DAY_OF_WEEK'].apply(get_day_name)

#do a line plot for all the delayed depatures by the day of the week
depatures_day_of_week_delays_df.set_index('DAY_NAME')['count'].plot(title='Delayed Depature Flights by Day of the Week', xlabel="Day of the Week", rot=45, color="green")


In [ ]:
#get all the delayed arrivals
delayed_arrivals=first_quarter_df[first_quarter_df['ARRIVAL_DELAY_STATUS']=='DELAYED']

#get day of week for all delayed arrivals
day_of_week_arrivals_df=pd.DataFrame(delayed_arrivals[["DAY_OF_WEEK"]])

#groub delayed arrivals and count them
day_of_week_delayed_arrivals_df=pd.DataFrame(day_of_week_arrivals_df.groupby(['DAY_OF_WEEK']).value_counts())

#reset the index
day_of_week_delayed_arrivals_df=day_of_week_delayed_arrivals_df.reset_index('DAY_OF_WEEK')



day_of_week_delayed_arrivals_df["DAY_NAME"]= day_of_week_delayed_arrivals_df['DAY_OF_WEEK'].apply(get_day_name)

day_of_week_delayed_arrivals_df.set_index('DAY_NAME')['count'].plot(title='Delayed Flight Arrivals by Day of the Week', xlabel="Day of the Week", rot=45)

In [ ]:
#get the top 9 airlines with the most depatures and arrivals. This filters out airlines with lower volume
airlines_to_study=pd.DataFrame(first_quarter_df['AIRLINE_CODE'].value_counts().head(9))

airlines_to_study=airlines_to_study.reset_index()

#function to get airline name from the airline code
def get_airline_name(iata_code):
    airline_name=''
    
    for index, row in airlines_df.iterrows():
        if row['IATA_CODE']==iata_code:
            airline_name=row['AIRLINE']
    
    return airline_name

#get airline name and assign it to the AIRLINE_NAME column
airlines_to_study['AIRLINE_NAME']=airlines_to_study['AIRLINE_CODE'].apply(get_airline_name)

#rename the count column to TOTAL_NUMBER_OF_FLIGHTS
airlines_to_study=airlines_to_study.rename(columns={"count": "TOTAL_NUMBER_OF_FLIGHTS"})

airlines_to_study=airlines_to_study.reset_index()

#reorder the columns
airlines_to_study = airlines_to_study[["AIRLINE_CODE","AIRLINE_NAME","TOTAL_NUMBER_OF_FLIGHTS"]]

airlines_to_study

In [ ]:
#create a list to filter out our data in our analysis
airline_filter_list= airlines_to_study['AIRLINE_CODE'].tolist()

airline_filter_list


In [ ]:
#filter and only get dataset for the top 9 high volume airlines
first_quarter_filtered_df=first_quarter_df[first_quarter_df['AIRLINE_CODE'].isin(airline_filter_list)] 

#get top 9 airlines with depature delays
airline_depatures_df = first_quarter_filtered_df[["AIRLINE_CODE","DEPATURE_DELAY_STATUS"]]

#get a count of depature delays grouped by airline
departures_by_airline=pd.DataFrame(airline_depatures_df.groupby(['AIRLINE_CODE','DEPATURE_DELAY_STATUS']).value_counts())

departures_by_airline=departures_by_airline.reset_index()

#Get actual airline name
departures_by_airline['AIRLINE_NAME']=departures_by_airline['AIRLINE_CODE'].apply(get_airline_name)
#departures_by_airline=departures_by_airline.set_index(['AIRLINE','DEPATURE_DELAY_STATUS'])

#rename count column
departures_by_airline=departures_by_airline.rename(columns={"count": "TOTAL_NUMBER_OF_FLIGHTS"})

#reorder the columns
departures_by_airline = departures_by_airline[["AIRLINE_CODE","AIRLINE_NAME","DEPATURE_DELAY_STATUS", "TOTAL_NUMBER_OF_FLIGHTS"]]

departures_by_airline


In [ ]:
#set index
departures_by_airline=departures_by_airline.set_index('AIRLINE_NAME', drop=True)

#departures_by_airline
depature_delays_df=departures_by_airline[departures_by_airline['DEPATURE_DELAY_STATUS']=='DELAYED']

#rename columns
depature_delays_df=depature_delays_df.rename(columns={'TOTAL_NUMBER_OF_FLIGHTS': 'TOTAL_DELAYED_FLIGHTS'})

depature_delays_df

In [ ]:
#draw a bar graph showing delays by airline
depature_delays_df.plot(kind="bar", figsize=(20,6), rot=45, title='Departure Delays for Top 9 Airlines', color="orange")

In [ ]:
#draw a pie chart

depature_delays_df.plot.pie(y='TOTAL_DELAYED_FLIGHTS', autopct='%1.1f%%', ylabel='', legend=False, rot=45)

In [ ]:
#get the top 9 airlines with the most flights
first_quarter_filtered_df=first_quarter_df[first_quarter_df['AIRLINE_CODE'].isin(airline_filter_list)] 

#drop rows with null arruvil delays
first_quarter_filtered_df.dropna(subset=['ARRIVAL_DELAY'], inplace=True)

#get airlines with their respective delay statuses
airline_arrivals_df = first_quarter_filtered_df[["AIRLINE_CODE","ARRIVAL_DELAY_STATUS"]]

airline_arrivals_df

In [ ]:
#get sum of arrivals statuses based on airline
arrivals_by_airline=pd.DataFrame(airline_arrivals_df.groupby(['AIRLINE_CODE','ARRIVAL_DELAY_STATUS']).value_counts())

#rename column
arrivals_by_airline=arrivals_by_airline.rename(columns={'count': 'NUMBER_OF_ARRIVALS'})

arrivals_by_airline=arrivals_by_airline.reset_index()

#use airline code to get airline name
arrivals_by_airline['AIRLINE_NAME']=arrivals_by_airline['AIRLINE_CODE'].apply(get_airline_name)

#create dataframe based needed columns
arrivals_by_airline = arrivals_by_airline[["AIRLINE_CODE","AIRLINE_NAME","ARRIVAL_DELAY_STATUS", "NUMBER_OF_ARRIVALS"]]

arrivals_by_airline


In [ ]:
#get all delayed arrival flights
arrival_delays_df=arrivals_by_airline[arrivals_by_airline['ARRIVAL_DELAY_STATUS']=='DELAYED']

#rename column
arrival_delays_df=arrival_delays_df.rename(columns={'NUMBER_OF_ARRIVALS': 'NUMBER_OF_DELAYED_ARRIVALS'})


arrival_delays_df=arrival_delays_df.set_index('AIRLINE_NAME')
arrival_delays_df.head(9)

In [ ]:
#plot number of delated arrivals by airline
arrival_delays_df.plot(kind="bar", figsize=(20,6), rot=45)

In [ ]:
#depature_delays_df.plot.(kind='pie', y='count')
arrival_delays_df.plot.pie(y='NUMBER_OF_DELAYED_ARRIVALS', ylabel='', autopct='%1.1f%%', legend=False, rot=45)

In [ ]:
delayed_flight_reasons=first_quarter_filtered_df[["AIRLINE_CODE", "AIR_SYSTEM_DELAY", "SECURITY_DELAY", "AIRLINE_DELAY", "LATE_AIRCRAFT_DELAY","WEATHER_DELAY"]]

#delayed_flight_reasons['AIR_SYSTEM_DELAY'] = delayed_flight_reasons.AIR_SYSTEM_DELAY.fillna(0)
delayed_flight_reasons.loc[:,"AIR_SYSTEM_DELAY"] = delayed_flight_reasons.AIR_SYSTEM_DELAY.fillna(0)
#delayed_flight_reasons['SECURITY_DELAY'] = delayed_flight_reasons.SECURITY_DELAY.fillna(0)
delayed_flight_reasons.loc[:,"SECURITY_DELAY"] = delayed_flight_reasons.SECURITY_DELAY.fillna(0)
#delayed_flight_reasons['AIRLINE_DELAY'] = delayed_flight_reasons.AIRLINE_DELAY.fillna(0)
delayed_flight_reasons.loc[:,"AIRLINE_DELAY"] = delayed_flight_reasons.AIRLINE_DELAY.fillna(0)
#delayed_flight_reasons['LATE_AIRCRAFT_DELAY'] = delayed_flight_reasons.LATE_AIRCRAFT_DELAY.fillna(0)
delayed_flight_reasons.loc[:,"LATE_AIRCRAFT_DELAY"] = delayed_flight_reasons.LATE_AIRCRAFT_DELAY.fillna(0)
#delayed_flight_reasons['WEATHER_DELAY'] = delayed_flight_reasons.WEATHER_DELAY.fillna(0)
delayed_flight_reasons.loc[:,"WEATHER_DELAY"] = delayed_flight_reasons.WEATHER_DELAY.fillna(0)

airline_delays_by_reason=delayed_flight_reasons.groupby(['AIRLINE_CODE'])[['AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY','AIR_SYSTEM_DELAY','WEATHER_DELAY','SECURITY_DELAY']].sum().reset_index()

#Get actual airline name
airline_delays_by_reason['AIRLINE_NAME']=airline_delays_by_reason['AIRLINE_CODE'].apply(get_airline_name)

airline_delays_by_reason=airline_delays_by_reason.drop(['AIRLINE_CODE'], axis=1)

airline_delays_by_reason=airline_delays_by_reason.set_index('AIRLINE_NAME')

airline_delays_by_reason=airline_delays_by_reason.sort_values(['AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY','AIR_SYSTEM_DELAY','WEATHER_DELAY','SECURITY_DELAY'], ascending=[False, False, False, False, False])

#airline_delays_by_reason = airline_delays_by_reason[["AIRLINE_NAME", "LATE_AIRCRAFT_DELAY","AIR_SYSTEM_DELAY","WEATHER_DELAY","SECURITY_DELAY"]]
airline_delays_by_reason


In [ ]:
airline_delays_by_reason[["AIR_SYSTEM_DELAY", "SECURITY_DELAY", "AIRLINE_DELAY", "LATE_AIRCRAFT_DELAY", "WEATHER_DELAY"]].plot(xlabel="Airline Name", ylabel='Delays by Millions of Minutes', figsize=(20,6), rot=45, lw=4)

In [ ]:
airline_delays_by_reason.plot.bar(figsize=(20,8),rot=45, xlabel="Airline Name",  ylabel='Delays by Millions of Minutes', width=1);